In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from models import TADW, TriDnr, DeepWalk, Node2Vec, Hope
from text_transformers import SBert, LDA, W2V, Sent2Vec, Doc2Vec, BOW, TFIDF
from datasets import Cora, CiteseerM10, Dblp

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from collections import defaultdict

from task import Task

[nltk_data] Downloading package wordnet to /Users/mikhail-
[nltk_data]     makarov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mikhail-makarov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/mikhail-
[nltk_data]     makarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
datasets = [
   ('Cora', Cora),
   # ('CiteseerM10', CiteseerM10),
   #  ('DBLP', Dblp)
]

test_ratios = [0.5, 0.7, 0.9, 0.95]

tasks = [
    # ('BOW', lambda ds: Task(ds, test_ratios, lambda: BOW(), None, d=None, labels=False)),
    # ('TFIDF', lambda ds: Task(ds, test_ratios, lambda: TFIDF(), None, d=None, labels=False)),
    # ('LDA', lambda ds: Task(ds, test_ratios, lambda: LDA(), None, d=None, labels=False)),
    # ('SBERT pretrained', lambda ds: Task(ds, test_ratios, lambda: SBert(train=False, d=300), None, d=None, labels=False)),
    # ('W2V pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=False, d=300), None, d=None, labels=False)),
    # ('W2V (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=300), None, d=None, labels=False)),
    # ('W2V (d=64)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), None, d=None, labels=False)),
    # ('Doc2Vec pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=False, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('Sent2Vec pretrained (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=False, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), None, d=None, labels=False)),
     ('TADW - BOW', lambda ds: Task(ds, test_ratios, BOW, TADW, d=160, labels=False)),
     ('TADW - TFIDF', lambda ds: Task(ds, test_ratios, TFIDF, TADW, d=160, labels=False)),
     ('TADW - Sent2Vec', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), TADW, d=160, labels=False)),
     ('TADW - Word2Vec', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), TADW, d=160, labels=False)),
]

In [3]:
res = {}

for ds_name, ds_constr in tqdm(datasets, desc='datasets'):
    ds = ds_constr()
    for task_name, task_constr in tqdm(tasks, desc='Tasks'):
        task = task_constr(ds)
        task_res = task.evaluate()
        for test_ratio in task_res:
            scores = task_res[test_ratio]
            res[f'{1 - test_ratio} - {ds_name} - {task_name}'] = scores

for name, scores in res.items():
    print(name, scores, np.mean(scores), np.std(scores))

Tasks:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19




test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  6.43it/s]


seeds:  40%|████      | 2/5 [00:00<00:00,  6.39it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  6.32it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00,  6.31it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00,  5.31it/s]




test_ratios:  25%|██▌       | 1/4 [00:00<00:02,  1.11it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  40%|████      | 2/5 [00:00<00:00, 10.76it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00, 10.89it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 11.10it/s]

test_ratios:  50%|█████     | 2/4 [00:01<00:01,  1.31it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 24.55it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 24.19it/s]

test_ratios:  75%|███████▌  | 3/4 [00:01<00:00,  1.67it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  80%|████████  | 

Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19




test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  6.60it/s]


seeds:  40%|████      | 2/5 [00:00<00:00,  6.86it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  7.17it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00,  7.46it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00,  7.56it/s]




test_ratios:  25%|██▌       | 1/4 [00:00<00:01,  1.51it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  40%|████      | 2/5 [00:00<00:00, 12.32it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00, 12.52it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 12.55it/s]

test_ratios:  50%|█████     | 2/4 [00:01<00:01,  1.71it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 21.71it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 21.48it/s]

test_ratios:  75%|███████▌  | 3/4 [00:01<00:00,  2.08it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 

Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19




test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  6.35it/s]


seeds:  40%|████      | 2/5 [00:00<00:00,  6.51it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  6.58it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00,  6.82it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00,  6.98it/s]




test_ratios:  25%|██▌       | 1/4 [00:00<00:02,  1.38it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  40%|████      | 2/5 [00:00<00:00, 11.76it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00, 11.77it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 11.82it/s]

test_ratios:  50%|█████     | 2/4 [00:01<00:01,  1.57it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 21.28it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 21.46it/s]

test_ratios:  75%|███████▌  | 3/4 [00:01<00:00,  1.94it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  80%|████████  | 

Iteration  0
Iteration  1
Iteration  2
Iteration  3
Iteration  4
Iteration  5
Iteration  6
Iteration  7
Iteration  8
Iteration  9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19




test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  6.69it/s]


seeds:  40%|████      | 2/5 [00:00<00:00,  6.68it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00,  6.85it/s]


seeds:  80%|████████  | 4/5 [00:00<00:00,  7.01it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00,  7.13it/s]




test_ratios:  25%|██▌       | 1/4 [00:00<00:02,  1.41it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  20%|██        | 1/5 [00:00<00:00,  9.48it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 10.18it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 10.78it/s]




test_ratios:  50%|█████     | 2/4 [00:01<00:01,  1.59it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  60%|██████    | 3/5 [00:00<00:00, 25.58it/s]


seeds: 100%|██████████| 5/5 [00:00<00:00, 25.71it/s]

test_ratios:  75%|███████▌  | 3/4 [00:01<00:00,  2.01it/s]


seeds:   0%|          | 0/5 [00:00<?, ?it/s]


seeds:  80%|████████ 

0.5 - Cora - TADW - BOW [0.8581979320531757, 0.8685376661742984, 0.8581979320531757, 0.8685376661742984, 0.8589364844903989] 0.8624815361890695 0.004952157936260037
0.30000000000000004 - Cora - TADW - BOW [0.8507383966244726, 0.8539029535864979, 0.8512658227848101, 0.8523206751054853, 0.8444092827004219] 0.8505274261603375 0.0032444212023691505
0.09999999999999998 - Cora - TADW - BOW [0.7986054142739952, 0.7977850697292863, 0.7871205906480722, 0.787940935192781, 0.7912223133716161] 0.79253486464315 0.004828199825943332
0.050000000000000044 - Cora - TADW - BOW [0.7143412359113875, 0.7162844928099493, 0.7182277497085114, 0.6859696851923824, 0.7438787407695298] 0.7157403808783521 0.018365741406880284
0.5 - Cora - TADW - TFIDF [0.8626292466765141, 0.8751846381093058, 0.8596750369276218, 0.864844903988183, 0.8545051698670606] 0.8633677991137372 0.006849053541725042
0.30000000000000004 - Cora - TADW - TFIDF [0.8470464135021097, 0.8697257383966245, 0.8544303797468354, 0.8502109704641351, 0.84